# 支付宝营销策略效果分析

A/B测试常用于比较不同设计、应用方案的优劣，以辅助决策。本次分析以支付宝营销活动为例，通过点击率指标比较两组营销策略的广澳投放效果。

## 1.数据来源

本文所使用的数据集来自阿里天池：

该数据集包含三张表，分别记录了支付宝两组营销策略的活动情况：

- emb_tb_2.csv:用户特征数据集
- effect_tb.csv:广告点击情况数据集
- seed_cand_tb.csv:用户类型数据集

本报告主要使用广澳点击情况数据，涉及字段如下：

- dmp_id:销售策略编号（源数据文档未作说明，这里根据数据情况设定为1：对照组；2：营销策略一；3：营销策略二）
- user_id:支付宝用户ID
- label:用户当天是否点击活动广告（0：未点击；1：点击）

## 2.数据处理

### （1）评估数据整齐度

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms

%matplotlib inline

In [2]:
#读取数据，并对列进行重命名
data = pd.read_csv('effect_tb.csv', header=None)
data.columns = ["dt","user_id","label","dmp_id"]

#删除不涉及分析主题的属性
data = data.drop(columns="dt")
data.head(10)

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2
3,1000006,0,1
4,1000006,0,3
5,1000007,0,1
6,1000008,0,3
7,1000014,0,1
8,1000016,0,3
9,1000018,0,1


数据整齐度无误

### （2）评估数据干净度

In [3]:
#查看数据摘要信息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645958 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   label    int64
 2   dmp_id   int64
dtypes: int64(3)
memory usage: 60.6 MB


从输出结果来看，“user_id” 列应该为字符串类型。

In [4]:
data['user_id'] = data['user_id'].astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645958 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   label    int64 
 2   dmp_id   int64 
dtypes: int64(2), object(1)
memory usage: 60.6+ MB


#### 【1】评估重复值

根据数据变量的含义来看，“user_id”是唯一标识符，不允许存在重复值

In [5]:
data.shape

(2645958, 3)

In [6]:
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

"user_id"作为唯一标识符，数据行数与独立用户数不统一，检查重复值

In [7]:
data[data.duplicated(keep=False)].sort_values(by = ["user_id"])

,user_id,label,dmp_id
215,1000625,0,1
1480321,1000625,0,1
356,1001026,0,1
1480397,1001026,0,1
600,1001855,0,1
...,...,...,...
1479589,998090,0,1
2645754,998892,0,1
1479854,998892,0,1
1480094,999610,0,1


In [8]:
#删除重复行
data = data.drop_duplicates()

In [9]:
data[data.duplicated(keep=False)]

,user_id,label,dmp_id


#### 【2】评估缺失值

In [10]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  2632975 non-null  object
 1   label    2632975 non-null  int64 
 2   dmp_id   2632975 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 80.4+ MB


数据集缺失值，无效进行处理

#### 【3】评估异常值

In [11]:
data.pivot_table(index="dmp_id",columns="label",values="user_id",
                 aggfunc="count",margins=True)

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


属性字段未发现缺失值，无需进行处理

## 3.样本容量检验

在进行A/B测试前，需检查样本容量是否满足实验所需最小值。

首先想要设定点击率基准线以及最小提升比例，我们将对照组的点击率设为基准线

In [12]:
data[data["dmp_id"]==1]["label"].mean()

0.012551012429794775

In [13]:
# 参数设置（根据第一张图数据）
baseline_rate = 0.0126  # 基准转化率1.26%
effect_size = 0.01      # 最小检测效果1个百分点
alpha = 0.05            # 显著性水平（默认5%）
power = 0.8             # 统计功效（默认80%）

# 计算样本量
effect = sms.proportion_effectsize(baseline_rate, baseline_rate + effect_size)
sample_size = sms.NormalIndPower().solve_power(
    effect_size=effect, 
    alpha=alpha, 
    power=power,
    ratio=1  # 两组样本相等
)

print(f"每组所需最小样本量：{round(sample_size)}")

每组所需最小样本量：2659


In [14]:
data["dmp_id"].value_counts()

1    1905663
2     411107
3     316205
Name: dmp_id, dtype: int64

两组营销活动的样本量分别为41，11万和31.62万，满足最小样本量需求

## 4.假设检验

先观察几组试验的点击率情况

In [15]:
print("对照组：",data[data["dmp_id"]==1]["label"].mean())
print("营销策略一：",data[data["dmp_id"]==2]["label"].mean())
print("营销策略二：",data[data["dmp_id"]==3]["label"].mean())

对照组： 0.012551012429794775
营销策略一： 0.015314747742072015
营销策略二： 0.026191869198779274


可以看到策略一和策略二相较对照组在点击率上都有不同程度的提升

其中策略一提升0.2个百分点，策略二提升1.3个百分点，只有策略二满足了前面我们对点击率提升最小值的要求

接下来需要进行假设检验，看策略二点击率的提升是否显著

__a.零假设和备择假设__

设对照组点击率为p1，策略二点击率为p2，则：

零假设 H0：p1 ≥ p2

备择假设 H1： p1 ≺ p2

__b.分布类型、检验类型和显著水平__

样本服从六点发布，独立双样本，样本大小n≻30，总体均值和标准差位置，所以采用Z检验。显著性水平ɑ取0.05

### （1）方法一：公式计算

In [17]:
#用户数
n_old=len(data[data.dmp_id==1]) #对照组
n_new=len(data[data.dmp_id==3]) #策略二

#点击数
c_old = len(data[(data.dmp_id==1) & (data.label==1)])
c_new = len(data[(data.dmp_id==3) & (data.label==1)])

#计算点击数
r_old=c_old/n_old
r_new=c_new/n_new

#总和点击率
r=(c_old+c_new)/(n_old+n_new)

print("总和点击率：",r)

总和点击率： 0.014492310074225832


In [20]:
#计算检验统计量z
z=(r_old-r_new)/np.sqrt(r*(1-r)*(1/n_old+1/n_new))

print("检验统计量Z：",z)

检验统计量Z： -59.44168632985996


In [21]:
#查ɑ=0.05对应的z分位数
from scipy.stats import norm
z_alpha=norm.ppf(0.05)
z_alpha

-1.6448536269514729

z=alpha=-1.64,检验统计量z=-59.44，该检验为左侧单位检验，拒绝域为{z<z_alpha}

所以我们可以得出结论：原假设不成立，策略二点击率的提升在统计上是显著的

### （2）方法二：Python函数计算

直接用pyhon statsmodels包计算z值和p值

In [24]:
import statsmodels.stats.proportion as sp

#进行比例z检验，备择假设为新策略组的点击率大于对照组
z_score,p=sp.proportions_ztest([c_old,c_new],[n_old,n_new],
                               alternative='smaller')

print("检验统计量z：",z_score,"，p值：",p)

检验统计量z： -59.44168632985996 ，p值： 0.0


p值约等于0，p<ɑ，与方法一结论相同，拒绝原假设

作为补充，我们再检验下策略一的点击率提升是否显著

In [30]:
#策略一检验
group2_click = len(data.query("dmp_id == 2 and label == 1"))
group1_total = len(data.query("dmp_id == 1"))
group2_total = len(data.query("dmp_id == 2"))

z_score, p = sp.proportions_ztest(
    [c_old, group2_click], 
    [group1_total, group2_total],
    alternative='smaller'
)
print("检验统计量z：", z_score, "，p值：", p)

检验统计量z： -14.165873564308429 ，p值： 7.450121742737582e-46


p值约等于7.45e-46，p<ɑ，但因为前面我们设置了对点击率提升的最小值要求，这里仍只选择第二组策略进行推广

## 5.结论

综上所述，这两种营销策略中，策略二对广告点击率有显著提升效果，且相较于对照组点击率提升了近一倍，因而在两组营销策略中应该选择第二组进行推广